In [ ]:
!pip install -U -q ultralytics
!pip install -U -q aifactory
!pip install -q tqdm

In [1]:
import os
from PIL import Image, ImageFile
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np
from tqdm import tqdm

ImageFile.LOAD_TRUNCATED_IMAGES = True  # 이미지 파일이 손상되었을 때 에러 발생 방지

def get_imglist(dir="./sample/img"):
    imglist = [os.path.join(dir, f).replace("\\", "/") for f in os.listdir(dir) if f.endswith('.png')]
    return imglist
import os
import shutil

def create_temp_dir(base_dir='./temp_cropped_patches'):
    # 임시 폴더가 이미 존재하면 삭제하고 새로 생성
    if os.path.exists(base_dir):
        shutil.rmtree(base_dir)
    os.makedirs(base_dir)
    return base_dir

def save_cropped_patches_single_image(image_path, crop_size, save_dir):
    image_name = os.path.basename(image_path)
    print(f"Cropping image: {image_name}")
    
    image = Image.open(image_path).convert('RGB')
    image_width, image_height = image.size
    
    num_patches_x = (image_width + crop_size - 1) // crop_size
    num_patches_y = (image_height + crop_size - 1) // crop_size
    
    for i in tqdm(range(num_patches_x)):
        for j in range(num_patches_y):
            top_left_x = i * crop_size
            top_left_y = j * crop_size
            bottom_right_x = min(top_left_x + crop_size, image_width)
            bottom_right_y = min(top_left_y + crop_size, image_height)
            
            if bottom_right_x - top_left_x < crop_size:
                top_left_x = max(image_width - crop_size, 0)
                bottom_right_x = image_width
            if bottom_right_y - top_left_y < crop_size:
                top_left_y = max(image_height - crop_size, 0)
                bottom_right_y = image_height
            
            cropped_image = image.crop((top_left_x, top_left_y, bottom_right_x, bottom_right_y))
            
            
            
            crop_filename = f"{os.path.splitext(image_name)[0]}_{top_left_x}_{top_left_y}.png"
            crop_path = os.path.join(save_dir, crop_filename)
            cropped_image.save(crop_path)
    
    del image
    
class CroppedPatchDataset(Dataset):
    def __init__(self, crop_image_dir,resize_size):
        self.crop_image_paths = crop_image_dir
        self.transform = transforms.ToTensor()
        self.resize_size = resize_size
    def __len__(self):
        return len(self.crop_image_paths)
    
    def __getitem__(self, idx):
        crop_image_path = self.crop_image_paths[idx]
        image_name = os.path.basename(crop_image_path)
        
        cropped_image = Image.open(crop_image_path).convert('RGB')
        
        cropped_image = cropped_image.resize((self.resize_size,self.resize_size), resample=Image.Resampling.LANCZOS)
        # BGR로 변환
        cropped_image_np = np.array(cropped_image)
        cropped_image_bgr = cropped_image_np[:, :, ::-1]
        cropped_image_pil = Image.fromarray(cropped_image_bgr, 'RGB')
        
        cropped_image_tensor = self.transform(cropped_image_pil)
        
        
        name_parts = image_name.split('_')
        top_left_x = int(name_parts[-2])
        top_left_y = int(name_parts[-1].split('.')[0])
        position = torch.tensor([top_left_x, top_left_y])
        
        original_image_name = '_'.join(name_parts[:-2]) + '.png'
        
        return {
            'image_name': original_image_name,
            'image': cropped_image_tensor,
            'top_left_position': position
        }


In [5]:
# from ultralytics import YOLO
# model = YOLO("yolo11s-obb.pt")
# preds = model.predict(images,conf=0.1, save=True)


0: 896x896 434.5ms
1: 896x896 (no detections), 434.5ms
2: 896x896 434.5ms
3: 896x896 (no detections), 434.5ms
4: 896x896 (no detections), 434.5ms
5: 896x896 (no detections), 434.5ms
6: 896x896 (no detections), 434.5ms
7: 896x896 (no detections), 434.5ms
8: 896x896 434.5ms
9: 896x896 (no detections), 434.5ms
10: 896x896 (no detections), 434.5ms
11: 896x896 (no detections), 434.5ms
12: 896x896 (no detections), 434.5ms
13: 896x896 (no detections), 434.5ms
14: 896x896 (no detections), 434.5ms
15: 896x896 434.5ms
16: 896x896 (no detections), 434.5ms
17: 896x896 (no detections), 434.5ms
18: 896x896 (no detections), 434.5ms
19: 896x896 434.5ms
20: 896x896 (no detections), 434.5ms
21: 896x896 (no detections), 434.5ms
22: 896x896 (no detections), 434.5ms
23: 896x896 (no detections), 434.5ms
24: 896x896 (no detections), 434.5ms
25: 896x896 (no detections), 434.5ms
26: 896x896 (no detections), 434.5ms
27: 896x896 434.5ms
28: 896x896 (no detections), 434.5ms
29: 896x896 (no detections), 434.5ms
3

In [8]:
# preds[27].obb.xywhr

tensor([[323.2016, 157.5453, 411.3317, 170.7461,   1.8658]])

In [9]:
# preds[27].obb.conf[0]

tensor([0.1042])

In [2]:
def run_model(image_names, images, positions, model, result):
    scale_factor = 0.25  # 예측 스케일과 패치 스케일의 비율: 256 / 1024

    preds = model.predict(images, conf=0.1, save=False)  # 예측 결과: [batch_size]

    for img_name, pred, pos in zip(image_names, preds, positions):
        if img_name not in result:
            result[img_name] = []

        top_left_x, top_left_y = pos[0].item(), pos[1].item()

        # 좌표 변환 및 conf 값 포함하여 저장
        for i in range(len(pred)):
            bbox = pred.obb.xywhr[i]  # [x, y, w, h, r]
            confidence = pred.obb.conf[i]  # confidence 값
            
            # bbox가 비어 있는 경우 해당 항목을 넘김
            if not bbox:
                continue

            # 예측된 좌표를 원본 이미지 좌표로 변환
            x = bbox[0].item() * scale_factor + top_left_x
            y = bbox[1].item() * scale_factor + top_left_y
            w = bbox[2].item() * scale_factor
            h = bbox[3].item() * scale_factor
            r = bbox[4].item()  # 각도 값은 변환 불필요

            # 변환된 좌표와 confidence를 결과 리스트에 추가
            result[img_name].append({
                'xywhr': [x, y, w, h, r],
                'conf': confidence.item()
            })


In [3]:
from ultralytics import YOLO
import torch
import shutil
from tqdm import tqdm
from ultralytics.utils.ops import nms_rotated

directory_path = '/workspace/dataset/'
# directory_path = './'
base_dir = './temp_cropped_patches'
img_list = get_imglist(directory_path)

# 모델 설정
model_name = "./runs/obb/combine_1024_0added/weights/best.pt"
crop_size = 256
batch_size = 16
model = YOLO(model_name)

result = {}  # 최종 결과를 저장할 딕셔너리

for image_path in img_list:
    # 임시 폴더 생성
    temp_dir = create_temp_dir(base_dir=base_dir)
    image_name = os.path.basename(image_path)
    print(f"Processing image: {image_name}")
    
    # 이미지 크롭 및 임시 폴더에 저장
    save_cropped_patches_single_image(image_path, crop_size, temp_dir)
    
    temp_list = get_imglist(temp_dir)
    # 데이터셋 및 DataLoader 설정
    dataset = CroppedPatchDataset(temp_list, resize_size=crop_size*4)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    # 원본 이미지별 결과 리스트 초기화
    result[image_name] = []
    
    for batch in tqdm(dataloader):
        images = batch['image']
        positions = batch['top_left_position']
        image_names = batch['image_name']
        
        # 모델 실행 및 결과 저장
        run_model(image_names, images, positions, model, result)
    
    # 현재 이미지의 모든 패치 예측을 NMS 적용
    per_image_result = result[image_name]
    if per_image_result:
        # boxes와 scores 추출
        boxes = torch.tensor([pred['xywhr'] for pred in per_image_result])
        scores = torch.tensor([pred['conf'] for pred in per_image_result])
        
        # NMS 적용
        keep_indices = nms_rotated(boxes, scores, threshold=0.45)
        
        # NMS 결과를 최종적으로 업데이트
        result[image_name] = [per_image_result[i] for i in keep_indices]

    # 임시 폴더 삭제
    shutil.rmtree(temp_dir)


Processing image: task_smaple.png
Cropping image: task_smaple.png


c:\ProgramData\anaconda3\envs\ship\Lib\site-packages\PIL\Image.py:3186: DecompressionBombWarning: Image size (120560400 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
  0%|          | 0/116 [00:00<?, ?it/s]


0: 1024x1024 (no detections), 47.4ms
1: 1024x1024 (no detections), 47.4ms
2: 1024x1024 47.4ms
3: 1024x1024 (no detections), 47.4ms
4: 1024x1024 (no detections), 47.4ms
5: 1024x1024 (no detections), 47.4ms
6: 1024x1024 (no detections), 47.4ms
7: 1024x1024 (no detections), 47.4ms
8: 1024x1024 (no detections), 47.4ms
9: 1024x1024 (no detections), 47.4ms
10: 1024x1024 (no detections), 47.4ms
11: 1024x1024 (no detections), 47.4ms
12: 1024x1024 (no detections), 47.4ms
13: 1024x1024 (no detections), 47.4ms
14: 1024x1024 (no detections), 47.4ms
15: 1024x1024 (no detections), 47.4ms
Speed: 1.9ms preprocess, 47.4ms inference, 10.1ms postprocess per image at shape (1, 3, 1024, 1024)


  1%|          | 1/116 [00:02<04:10,  2.18s/it]


0: 1024x1024 (no detections), 38.9ms
1: 1024x1024 (no detections), 38.9ms
2: 1024x1024 (no detections), 38.9ms
3: 1024x1024 (no detections), 38.9ms
4: 1024x1024 (no detections), 38.9ms
5: 1024x1024 (no detections), 38.9ms
6: 1024x1024 (no detections), 38.9ms
7: 1024x1024 (no detections), 38.9ms
8: 1024x1024 (no detections), 38.9ms
9: 1024x1024 (no detections), 38.9ms
10: 1024x1024 (no detections), 38.9ms
11: 1024x1024 38.9ms
12: 1024x1024 (no detections), 38.9ms
13: 1024x1024 (no detections), 38.9ms
14: 1024x1024 (no detections), 38.9ms
15: 1024x1024 (no detections), 38.9ms
Speed: 1.5ms preprocess, 38.9ms inference, 6.7ms postprocess per image at shape (1, 3, 1024, 1024)


...

 98%|█████████▊| 114/116 [02:26<00:02,  1.28s/it]


0: 1024x1024 39.7ms
1: 1024x1024 (no detections), 39.7ms
2: 1024x1024 (no detections), 39.7ms
3: 1024x1024 (no detections), 39.7ms
4: 1024x1024 (no detections), 39.7ms
5: 1024x1024 (no detections), 39.7ms
6: 1024x1024 (no detections), 39.7ms
7: 1024x1024 (no detections), 39.7ms
8: 1024x1024 (no detections), 39.7ms
9: 1024x1024 39.7ms
10: 1024x1024 (no detections), 39.7ms
11: 1024x1024 (no detections), 39.7ms
12: 1024x1024 (no detections), 39.7ms
13: 1024x1024 (no detections), 39.7ms
14: 1024x1024 (no detections), 39.7ms
15: 1024x1024 (no detections), 39.7ms
Speed: 1.7ms preprocess, 39.7ms inference, 5.9ms postprocess per image at shape (1, 3, 1024, 1024)


 99%|█████████▉| 115/116 [02:27<00:01,  1.28s/it]


0: 1024x1024 (no detections), 38.6ms
1: 1024x1024 (no detections), 38.6ms
2: 1024x1024 (no detections), 38.6ms
3: 1024x1024 (no detections), 38.6ms
4: 1024x1024 (no detections), 38.6ms
5: 1024x1024 (no detections), 38.6ms
6: 1024x1024 (no detections), 38.6ms
7: 1024x1024 (no detections), 38.6ms
8: 1024x1024 (no detections), 38.6ms
Speed: 1.5ms preprocess, 38.6ms inference, 7.6ms postprocess per image at shape (1, 3, 1024, 1024)


100%|██████████| 116/116 [02:28<00:00,  1.28s/it]


result = 
{'task_smaple.png': [{'xywhr': [tensor(229.2828, device='cuda:0'),
    tensor(10279.5654, device='cuda:0'),
    tensor(82.1188, device='cuda:0'),
    tensor(14.0488, device='cuda:0'),
    tensor(1.9125, device='cuda:0')],
   'conf': 0.34337615966796875},
  {'xywhr': [tensor(151.2560, device='cuda:0'),
    tensor(10446.5742, device='cuda:0'),
    tensor(110.7664, device='cuda:0'),
    tensor(25.3042, device='cuda:0'),
    tensor(2.5097, device='cuda:0')],
   'conf': 0.27472221851348877},
   ...]}


In [5]:
import csv
import math

# 저장할 CSV 경로
csv_file = "./submission.csv"
data = []  # CSV에 저장할 데이터를 담을 리스트

# 이미지 이름별로 데이터 변환
for image_name, predictions in result.items():
    if not predictions:
        continue
    
    # 각 예측 결과를 변환하여 data 리스트에 추가
    for pred in predictions:
        cx, cy, width, height, angle = pred['xywhr']
        
        # 각도 변환: 라디안 -> 도(degrees)
        angle_deg = math.degrees(angle)
        if angle_deg < 0:
            angle_deg += 360
        
        # 예측 결과를 리스트로 추가
        data.append([image_name, cx.item(), cy.item(), width.item(), height.item(), angle_deg])

# CSV 파일로 저장
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # CSV의 헤더 작성
    writer.writerow(['image_name', 'cx', 'cy', 'width', 'height', 'angle'])
    
    # 각 행을 작성
    writer.writerows(data)

print(f"CSV 파일 '{csv_file}'이(가) 성공적으로 생성되었습니다.")


CSV 파일 './submission.csv'이(가) 성공적으로 생성되었습니다.


In [2]:
import aifactory.score as aif
import time

t = time.time()
aif.submit(model_name="combine_1024_0added_1_task_resized",
           key="246f41b0-c912-46f5-8f9a-42171aa1f7f0")
print("time:", time.time() - t)

file : task.py
python
제출 완료
time: 85.56951189041138
